# Compaction 전략 실습 — Binpack, Sort, Z-order

이 노트북에서는 Iceberg의 **Compaction(컴팩션)** 개념과 3가지 전략(Binpack, Sort, Z-order)을 실습합니다.

## Compaction이란?

Compaction(컴팩션)은 **여러 개의 작은 파일을 더 적은 수의 큰 파일로 재작성**하는 프로세스입니다.

### 왜 필요한가?

데이터를 읽을 때 두 가지 비용이 발생합니다:
- **고정 비용**: 쿼리에 필요한 데이터를 실제로 읽는 비용 (피할 수 없음)
- **가변 비용**: 파일을 열고, 스캔하고, 닫는 파일 작업 비용 (줄일 수 있음!)

파일이 많을수록 가변 비용이 증가합니다. 특히 스트리밍 환경에서는 각각 몇 개의 레코드만 담은 수많은 작은 파일이 생성되어 **Small File Problem**이 심각해집니다.

### 3가지 컴팩션 전략

| 전략 | 동작 | 장점 | 단점 |
|------|------|------|------|
| **Binpack** | 파일만 결합 (정렬 없음) | 가장 빠름 | 데이터 순서 변경 없음 |
| **Sort** | 지정 필드로 순차 정렬 후 결합 | 단일 필드 프루닝 가능 | Binpack보다 느림 |
| **Z-order** | 여러 필드를 동등 가중치로 정렬 | 복합 필터 프루닝 가능 | Binpack보다 느림 |

## 환경 설정

In [1]:
import sys
sys.path.append('..')

import time
from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [2]:
spark = create_spark_session()

# 테이블 설정
TABLE_NAME = "demo.lab.compact_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_orders"

SORT_TABLE = "demo.lab.compact_sort_orders"
SORT_TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_sort_orders"

ZORDER_TABLE = "demo.lab.compact_zorder_orders"
ZORDER_TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_zorder_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


---
## 실험 1: Small File Problem 시연

10건씩 10번 반복 INSERT하여 작은 파일이 폭증하는 현상을 관찰합니다.

In [3]:
# Binpack 실험용 테이블 생성
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

print(f"테이블 생성 완료: {TABLE_NAME}")

테이블 생성 완료: demo.lab.compact_orders


In [4]:
# 10건씩 10번 반복 INSERT → 작은 파일 폭증
for i in range(10):
    orders = generate_orders(num_records=10, seed=i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(TABLE_NAME).append()
    print(f"배치 {i+1}/10 완료 (10건 삽입)")

print(f"\n총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

배치 1/10 완료 (10건 삽입)
배치 2/10 완료 (10건 삽입)
배치 3/10 완료 (10건 삽입)
배치 4/10 완료 (10건 삽입)
배치 5/10 완료 (10건 삽입)
배치 6/10 완료 (10건 삽입)
배치 7/10 완료 (10건 삽입)
배치 8/10 완료 (10건 삽입)
배치 9/10 완료 (10건 삽입)
배치 10/10 완료 (10건 삽입)

총 레코드 수: 100


In [5]:
# Small File Problem 확인
print("파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

parquet_count = count_files(TABLE_PATH)
total = total_size(TABLE_PATH)
print(f"\nParquet 파일 수: {parquet_count}")
print(f"총 크기: {total:,} bytes")

파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-11-d5e32a30-af27-4857-a07e-ee62a8ae10ef-00001.parquet  (1.9 KB)
│   │   ├── 00000-17-ddc4dd2c-3bda-4d2c-8f67-d27da792f2e7-00001.parquet  (1.9 KB)
│   │   ├── 00000-23-9fd64224-1805-4829-b8a2-1a0afff47950-00001.parquet  (1.9 KB)
│   │   ├── 00000-29-34d94f64-7ad3-4385-90f5-f199623b6f42-00001.parquet  (1.8 KB)
│   │   ├── 00000-35-1d5224a4-42e5-4b41-9e0e-b62752e5110c-00001.parquet  (2.0 KB)
│   │   ├── 00000-41-0f25a567-19a9-4c51-9165-68ed387c39a1-00001.parquet  (1.7 KB)
│   │   ├── 00000-47-69e5fddf-e275-4d9e-9b58-700e3163181c-00001.parquet  (2.0 KB)
│   │   ├── 00000-5-357b99e2-4df0-45cf-943f-112bd954656b-00001.parquet  (1.8 KB)
│   │   ├── 00000-53-12ad42cd-5871-4ad4-9bbd-a464d9fa3854-00001.parquet  (1.9 KB)
│   │   └── 00000-59-a6d65912-2926-4fa2-8e3f-9105ec6e5c53-00001.parquet  (1.8 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-11-d5e32a30-af27-4857-a07e-ee62a8ae10ef-00002.parquet  (1.8 KB)
│   │   ├── 00

In [6]:
# 쿼리 성능 측정 (컴팩션 전)
start = time.time()
spark.sql(f"SELECT * FROM {TABLE_NAME}").collect()
elapsed_before = time.time() - start
active_data_files_before = spark.sql(f"SELECT COUNT(*) AS cnt FROM {TABLE_NAME}.files WHERE content = 0").collect()[0]['cnt']
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 전): {elapsed_before:.3f}초")
print(f"현재 스냅샷 기준 스캔 대상 데이터 파일 수: {active_data_files_before}")
print(f"디스크 상 Parquet 파일 수(히스토리 포함): {parquet_count}")

전체 테이블 SELECT 소요 시간 (컴팩션 전): 0.582초
현재 스냅샷 기준 스캔 대상 데이터 파일 수: 30
디스크 상 Parquet 파일 수(히스토리 포함): 30


### 관찰 포인트 — Small File Problem

- 10번의 INSERT로 각 파티션마다 최대 10개의 작은 파일이 생성되었습니다
- 100건의 작은 데이터에 비해 파일 수가 과도하게 많습니다
- 이는 스트리밍 수집이나 빈번한 소량 INSERT에서 흔히 발생하는 문제입니다

---
## 실험 2: Binpack 전략

Binpack은 가장 빠른 컴팩션 전략으로, **데이터 순서를 변경하지 않고 파일만 결합**합니다.

In [7]:
before = snapshot_tree(TABLE_PATH)

# Binpack 컴팩션 실행
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{TABLE_NAME}',
    strategy => 'binpack',
    options => map('target-file-size-bytes', '134217728')
)
""").show(truncate=False)

after = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("Binpack 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|30                        |3                     |57169                |0                      |
+--------------------------+----------------------+---------------------+-----------------------+

Binpack 컴팩션 전후 변경 사항:

[+] 추가된 파일 (16개):
    + data/order_date_month=2024-01/00000-76-c5082cdc-3eb7-454d-8d22-7efa3d1436fb-00001.parquet  (2.5 KB)
    + data/order_date_month=2024-02/00000-75-eea4ab25-53a4-4ee6-a935-6d12187b6a3f-00001.parquet  (2.5 KB)
    + data/order_date_month=2024-03/00000-74-dc825c8e-626f-4b36-abaa-fadf58c8c9aa-00001.parquet  (2.4 KB)
    + metadata/b3eae3be-32d0-4fa6-aa74-93423493c6ac-m0.avro  (7.3 KB)
    + metadata/b3eae3be-32d0-4fa6-aa74-93423493c6ac-m1.avro  (7.3 KB)
    + meta

In [8]:
# 컴팩션 후 파일 구조
print("Binpack 후 파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

new_parquet_count = count_files(TABLE_PATH)
new_active_data_files = spark.sql(f"SELECT COUNT(*) AS cnt FROM {TABLE_NAME}.files WHERE content = 0").collect()[0]['cnt']
new_total = total_size(TABLE_PATH)
print(f"\n디스크 상 Parquet 파일 수(히스토리 포함): {parquet_count} → {new_parquet_count}")
print(f"현재 스냅샷 기준 스캔 대상 데이터 파일 수: {active_data_files_before} → {new_active_data_files}")
print(f"총 크기: {total:,} → {new_total:,} bytes")
print("참고: 기존 파일은 삭제 마킹만 되고, 스냅샷 만료/고아 파일 정리 전까지 디스크에 남을 수 있습니다.")

Binpack 후 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-11-d5e32a30-af27-4857-a07e-ee62a8ae10ef-00001.parquet  (1.9 KB)
│   │   ├── 00000-17-ddc4dd2c-3bda-4d2c-8f67-d27da792f2e7-00001.parquet  (1.9 KB)
│   │   ├── 00000-23-9fd64224-1805-4829-b8a2-1a0afff47950-00001.parquet  (1.9 KB)
│   │   ├── 00000-29-34d94f64-7ad3-4385-90f5-f199623b6f42-00001.parquet  (1.8 KB)
│   │   ├── 00000-35-1d5224a4-42e5-4b41-9e0e-b62752e5110c-00001.parquet  (2.0 KB)
│   │   ├── 00000-41-0f25a567-19a9-4c51-9165-68ed387c39a1-00001.parquet  (1.7 KB)
│   │   ├── 00000-47-69e5fddf-e275-4d9e-9b58-700e3163181c-00001.parquet  (2.0 KB)
│   │   ├── 00000-5-357b99e2-4df0-45cf-943f-112bd954656b-00001.parquet  (1.8 KB)
│   │   ├── 00000-53-12ad42cd-5871-4ad4-9bbd-a464d9fa3854-00001.parquet  (1.9 KB)
│   │   ├── 00000-59-a6d65912-2926-4fa2-8e3f-9105ec6e5c53-00001.parquet  (1.8 KB)
│   │   └── 00000-76-c5082cdc-3eb7-454d-8d22-7efa3d1436fb-00001.parquet  (2.5 KB)
│   ├── order_date_month=2024-02/
│   

In [9]:
# 쿼리 성능 재측정
start = time.time()
spark.sql(f"SELECT * FROM {TABLE_NAME}").collect()
elapsed_after = time.time() - start
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 전): {elapsed_before:.3f}초")
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 후): {elapsed_after:.3f}초")
print(f"현재 스냅샷 기준 스캔 대상 데이터 파일 수: {active_data_files_before} → {new_active_data_files}")
print(f"디스크 상 Parquet 파일 수(히스토리 포함): {parquet_count} → {new_parquet_count}")

전체 테이블 SELECT 소요 시간 (컴팩션 전): 0.582초
전체 테이블 SELECT 소요 시간 (컴팩션 후): 0.106초
현재 스냅샷 기준 스캔 대상 데이터 파일 수: 30 → 3
디스크 상 Parquet 파일 수(히스토리 포함): 30 → 33


### 관찰 포인트 — Binpack

- **현재 스냅샷 기준 스캔 대상 데이터 파일 수는 30 → 3으로 감소**했습니다 (파티션당 1개로 통합)
- **디스크 상 Parquet 개수는 30 → 33으로 보일 수 있습니다** (기존 30개 + 새 3개, 기존 파일은 즉시 물리 삭제되지 않음)
- 데이터 순서는 변경되지 않았습니다 — 단순히 파일만 합쳤습니다
- Binpack은 **가장 빠른 전략**으로, 스트리밍 SLA를 충족해야 할 때 적합합니다
- 쿼리는 현재 스냅샷의 활성 파일만 읽으므로 파일 작업(open/scan/close) 비용이 줄어듭니다
- 불필요한 물리 파일 정리는 `expire_snapshots` + `remove_orphan_files`를 통해 수행합니다

---
## 실험 3: Sort 전략

Sort 전략은 데이터를 **지정 필드로 정렬한 후 결합**합니다. 정렬하면 유사한 값이 적은 파일에 집중되어, manifest의 min/max 통계를 통한 **파일 프루닝**이 가능해집니다.

In [10]:
# Sort 실험용 테이블 생성 + 데이터 삽입
spark.sql(f"DROP TABLE IF EXISTS {SORT_TABLE}")

spark.sql(f"""
CREATE TABLE {SORT_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 10건씩 10번 반복 INSERT (작은 파일 생성)
for i in range(10):
    orders = generate_orders(num_records=10, seed=100+i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(SORT_TABLE).append()

print(f"테이블 생성 + 100건 삽입 완료: {SORT_TABLE}")
print(f"Parquet 파일 수: {count_files(SORT_TABLE_PATH)}")

테이블 생성 + 100건 삽입 완료: demo.lab.compact_sort_orders
Parquet 파일 수: 30


In [11]:
before = snapshot_tree(SORT_TABLE_PATH)

# Sort 컴팩션 실행 (product_name 기준 정렬)
# 데모 가시성을 위해 target-file-size를 작게 설정하여 파일 범위(min/max)가 더 잘 드러나게 함
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{SORT_TABLE}',
    strategy => 'sort',
    sort_order => 'product_name ASC NULLS LAST',
    options => map('target-file-size-bytes', '8192')
)
""").show(truncate=False)

after = snapshot_tree(SORT_TABLE_PATH)

print("=" * 60)
print("Sort 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|30                        |9                     |57205                |0                      |
+--------------------------+----------------------+---------------------+-----------------------+

Sort 컴팩션 전후 변경 사항:

[+] 추가된 파일 (22개):
    + data/order_date_month=2024-01/00000-154-e3a8216c-f9ba-4bf0-aeb8-92229574edb7-00001.parquet  (2.1 KB)
    + data/order_date_month=2024-01/00001-155-e3a8216c-f9ba-4bf0-aeb8-92229574edb7-00001.parquet  (2.0 KB)
    + data/order_date_month=2024-01/00002-156-e3a8216c-f9ba-4bf0-aeb8-92229574edb7-00001.parquet  (2.1 KB)
    + data/order_date_month=2024-02/00000-149-ed72f360-af5c-46b2-88ea-99d692104f64-00001.parquet  (2.2 KB)
    + data/order_date_month=2024-02/00001-1

In [12]:
# Sort 후 파일 구조 확인
print("Sort 후 파일 구조:")
print("=" * 60)
show_tree(SORT_TABLE_PATH)

print(f"\nParquet 파일 수: {count_files(SORT_TABLE_PATH)}")

Sort 후 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-100-75db31f7-fcf1-4ad4-8434-500fba2d2dc0-00001.parquet  (1.8 KB)
│   │   ├── 00000-106-24894dd4-6f89-4b96-8894-c46abe289d0b-00001.parquet  (1.9 KB)
│   │   ├── 00000-112-600e08ab-18e6-4a3c-b597-947be3458de4-00001.parquet  (1.9 KB)
│   │   ├── 00000-118-c8135d94-4776-47d5-af5e-b09ebfee8e69-00001.parquet  (1.8 KB)
│   │   ├── 00000-124-9cda1510-d92e-4b7c-b4b5-1d141c2d257c-00001.parquet  (1.8 KB)
│   │   ├── 00000-130-2ab75157-5bf7-4e70-9e9a-9b1a832c64fb-00001.parquet  (1.9 KB)
│   │   ├── 00000-136-931717fc-0838-4664-948c-d10f9d7cbd84-00001.parquet  (1.9 KB)
│   │   ├── 00000-142-da25ba9e-1bf7-4fa2-b194-501ecff10aea-00001.parquet  (1.8 KB)
│   │   ├── 00000-154-e3a8216c-f9ba-4bf0-aeb8-92229574edb7-00001.parquet  (2.1 KB)
│   │   ├── 00000-88-da75732a-8deb-403b-8b27-e71b11f90650-00001.parquet  (1.9 KB)
│   │   ├── 00000-94-f5d0f064-837e-449e-859e-e9f2b92d7f47-00001.parquet  (1.9 KB)
│   │   ├── 00001-155-e3a8216c-

In [13]:
# 1) manifests 확인 — partition_summaries의 lower/upper는 파티션 컬럼(order_date_month) 범위
print("Manifest 파일 정보 (partition_summaries는 파티션 범위):")
spark.sql(f"""
SELECT
    path,
    added_data_files_count,
    existing_data_files_count,
    deleted_data_files_count,
    partition_summaries
FROM {SORT_TABLE}.manifests
""").show(truncate=False)

# 2) files 확인 — 파일별 product_name min/max (field id=3)
print("\n파일별 product_name min/max (active data files only):")
spark.sql(f"""
SELECT
    regexp_replace(file_path, '^file:.*?/(data/)', '$1') AS file_path,
    record_count,
    CAST(lower_bounds[3] AS STRING) AS product_name_min,
    CAST(upper_bounds[3] AS STRING) AS product_name_max
FROM {SORT_TABLE}.files
WHERE content = 0
ORDER BY product_name_min, product_name_max
""").show(truncate=False)

# 3) 파티션 내 인접 파일 범위 관계 확인 — overlap이 줄었는지 확인
print("\n파티션 내 인접 파일 간 범위 관계:")
spark.sql(f"""
WITH ranges AS (
    SELECT
        regexp_replace(file_path, '^file:.*?/(data/)', '$1') AS file_path,
        regexp_extract(file_path, 'order_date_month=([^/]+)', 1) AS order_date_month,
        CAST(lower_bounds[3] AS STRING) AS product_name_min,
        CAST(upper_bounds[3] AS STRING) AS product_name_max
    FROM {SORT_TABLE}.files
    WHERE content = 0
),
ordered AS (
    SELECT
        file_path,
        order_date_month,
        product_name_min,
        product_name_max,
        LAG(product_name_max) OVER (
            PARTITION BY order_date_month
            ORDER BY product_name_min, product_name_max, file_path
        ) AS prev_file_max
    FROM ranges
)
SELECT
    order_date_month,
    file_path,
    product_name_min,
    product_name_max,
    prev_file_max,
    CASE
        WHEN prev_file_max IS NULL THEN 'first-in-partition'
        WHEN prev_file_max <= product_name_min THEN 'non-overlap'
        ELSE 'overlap'
    END AS range_relation
FROM ordered
ORDER BY order_date_month, product_name_min, product_name_max, file_path
""").show(truncate=False)

Manifest 파일 정보 (partition_summaries는 파티션 범위):
+---------------------------------------------------------------------------------------------------------------+----------------------+-------------------------+------------------------+----------------------------------+
|path                                                                                                           |added_data_files_count|existing_data_files_count|deleted_data_files_count|partition_summaries               |
+---------------------------------------------------------------------------------------------------------------+----------------------+-------------------------+------------------------+----------------------------------+
|file:/home/jovyan/data/warehouse/lab/compact_sort_orders/metadata/22783cfd-742a-4c6d-8fb5-90c83f0032f2-m10.avro|9                     |0                        |0                       |[{false, false, 2024-01, 2024-03}]|
|file:/home/jovyan/data/warehouse/lab/compact_sort_orders/meta

In [14]:
# 특정 product_name으로 필터링 — EXPLAIN에서 볼 포인트 + 파일 프루닝 추정
target_product = "iPhone 14"

print(f"EXPLAIN: product_name = '{target_product}' 필터 쿼리")
print("- 볼 포인트: Physical Plan의 BatchScan filters에 predicate pushdown이 반영됐는지")
spark.sql(f"""
EXPLAIN FORMATTED
SELECT *
FROM {SORT_TABLE}
WHERE product_name = '{target_product}'
""").show(truncate=False)

# EXPLAIN에는 파일 개수가 직접 표시되지 않을 수 있으므로 .files 메타데이터로 보완
prune_stats = spark.sql(f"""
SELECT
    COUNT(*) AS total_active_files,
    SUM(CASE
        WHEN CAST(lower_bounds[3] AS STRING) <= '{target_product}'
         AND CAST(upper_bounds[3] AS STRING) >= '{target_product}'
        THEN 1 ELSE 0 END) AS candidate_files
FROM {SORT_TABLE}.files
WHERE content = 0
""").collect()[0]

print(f"\n파일 프루닝 추정: 전체 {prune_stats['total_active_files']}개 중 후보 {prune_stats['candidate_files']}개")
print(f"예상 prune 파일 수: {prune_stats['total_active_files'] - prune_stats['candidate_files']}개")

EXPLAIN: product_name = 'iPhone 14' 필터 쿼리
- 볼 포인트: Physical Plan의 BatchScan filters에 predicate pushdown이 반영됐는지
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|plan                                                                                                                           

### 관찰 포인트 — Sort

- 데이터가 `product_name` 기준으로 **정렬되어 재작성**되었습니다
- `{SORT_TABLE}.manifests`의 `partition_summaries` lower/upper는 **파티션 컬럼(order_date_month)** 범위입니다
- `product_name` 파일별 min/max는 `{SORT_TABLE}.files`의 `lower_bounds[3]`/`upper_bounds[3]`에서 확인합니다 (`content=0`만 조회)
- 데모에서는 `target-file-size-bytes=8192`로 출력 파일 수를 늘려 정렬 효과(min/max 분리)를 더 잘 보이게 했습니다
- 정렬 후 각 파일의 `product_name` min/max 범위가 좁아지고, **파티션 내 인접 파일** 간 overlap이 줄어들수록 파일 프루닝에 유리합니다
- EXPLAIN에서는 `BatchScan ... [filters=...]`로 predicate pushdown 여부를 확인하고, 파일 개수는 `.files` 통계로 함께 확인해야 합니다
- Sort는 **자주 필터링하는 단일 필드**가 있을 때 가장 효과적입니다

---
## 실험 4: Z-order 전략

Z-order는 **여러 필드를 동등한 가중치로 다차원 정렬**합니다.

### Z-order 개념

일반 정렬은 1차원적입니다 — A 기준으로 정렬하면 B는 순서가 보장되지 않습니다.

Z-order는 **사분면 기반 탐색**을 사용합니다:
1. X, Y 두 축의 값 범위를 4개 사분면으로 나눕니다
2. 검색값의 사분면을 확인하면, 나머지 3개 사분면(75%!)을 즉시 제외할 수 있습니다
3. 사분면을 재귀적으로 세분화하여 검색 범위를 좁힙니다

이 방식은 `WHERE product_name = 'X' AND customer_id = 123`처럼 **복합 필터 쿼리**에서 큰 효과를 발휘합니다.

In [15]:
# Z-order 실험용 테이블 생성 + 데이터 삽입
spark.sql(f"DROP TABLE IF EXISTS {ZORDER_TABLE}")

spark.sql(f"""
CREATE TABLE {ZORDER_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 10건씩 10번 반복 INSERT
for i in range(10):
    orders = generate_orders(num_records=10, seed=200+i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(ZORDER_TABLE).append()

print(f"테이블 생성 + 100건 삽입 완료: {ZORDER_TABLE}")
print(f"Parquet 파일 수: {count_files(ZORDER_TABLE_PATH)}")

테이블 생성 + 100건 삽입 완료: demo.lab.compact_zorder_orders
Parquet 파일 수: 30


In [16]:
before = snapshot_tree(ZORDER_TABLE_PATH)

# Z-order 컴팩션 실행 (product_name + customer_id 기준)
# 데모 가시성을 위해 target-file-size를 작게 설정하여 복합 필터 프루닝 효과를 관찰하기 쉽게 함
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{ZORDER_TABLE}',
    strategy => 'sort',
    sort_order => 'zorder(product_name, customer_id)',
    options => map('target-file-size-bytes', '8192')
)
""").show(truncate=False)

after = snapshot_tree(ZORDER_TABLE_PATH)

print("=" * 60)
print("Z-order 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|30                        |9                     |57072                |0                      |
+--------------------------+----------------------+---------------------+-----------------------+

Z-order 컴팩션 전후 변경 사항:

[+] 추가된 파일 (22개):
    + data/order_date_month=2024-01/00000-243-b32107bf-9bdb-4093-8813-eebbb3de8628-00001.parquet  (2.1 KB)
    + data/order_date_month=2024-01/00001-244-b32107bf-9bdb-4093-8813-eebbb3de8628-00001.parquet  (2.1 KB)
    + data/order_date_month=2024-01/00002-245-b32107bf-9bdb-4093-8813-eebbb3de8628-00001.parquet  (2.0 KB)
    + data/order_date_month=2024-02/00000-240-1bfd9e1c-7e0a-4f77-8202-e5caa792d3dd-00001.parquet  (2.1 KB)
    + data/order_date_month=2024-02/0000

In [17]:
# Z-order 후 파일 구조 확인
print("Z-order 후 파일 구조:")
print("=" * 60)
show_tree(ZORDER_TABLE_PATH)

z_active_files = spark.sql(f"SELECT COUNT(*) AS cnt FROM {ZORDER_TABLE}.files WHERE content = 0").collect()[0]['cnt']
z_parquet_files = count_files(ZORDER_TABLE_PATH)
print(f"\n현재 스냅샷 기준 스캔 대상 데이터 파일 수: {z_active_files}")
print(f"디스크 상 Parquet 파일 수(히스토리 포함): {z_parquet_files}")

Z-order 후 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-176-06aa140a-208a-4bd9-9a46-7e705dfc25f3-00001.parquet  (1.8 KB)
│   │   ├── 00000-182-0f1c7bca-70de-42c3-b104-4793eb2c4d21-00001.parquet  (1.8 KB)
│   │   ├── 00000-188-39dc9007-d1a4-47ff-ac1c-39a36553eb9b-00001.parquet  (2.0 KB)
│   │   ├── 00000-194-8d84f97d-9426-4911-87bd-ef81956cdb82-00001.parquet  (1.9 KB)
│   │   ├── 00000-200-e4ac2205-16af-4073-93e1-3161a9d4b694-00001.parquet  (1.8 KB)
│   │   ├── 00000-206-dcd1a3d3-84eb-45c5-8164-9ddb3212a28e-00001.parquet  (1.9 KB)
│   │   ├── 00000-212-d7939634-0003-4186-923c-42c3019b0c33-00001.parquet  (1.9 KB)
│   │   ├── 00000-218-753ff9ee-cba5-4178-a873-7461594ffca4-00001.parquet  (1.8 KB)
│   │   ├── 00000-224-f7904878-796d-4958-9c8e-83d558b780f9-00001.parquet  (1.9 KB)
│   │   ├── 00000-230-bc308ce7-1ce4-4f1a-b53a-f48958cc173d-00001.parquet  (1.9 KB)
│   │   ├── 00000-243-b32107bf-9bdb-4093-8813-eebbb3de8628-00001.parquet  (2.1 KB)
│   │   ├── 00001-244-b321

In [18]:
# 복합 필터 쿼리의 프루닝 효과 확인
print("EXPLAIN: product_name + customer_id 복합 필터 쿼리")
print("- 볼 포인트: Physical Plan의 BatchScan filters에 두 조건이 모두 포함되는지")
spark.sql(f"""
EXPLAIN FORMATTED
SELECT * FROM {ZORDER_TABLE}
WHERE product_name = 'iPhone 14' AND customer_id = 500
""").show(truncate=False)

EXPLAIN: product_name + customer_id 복합 필터 쿼리
- 볼 포인트: Physical Plan의 BatchScan filters에 두 조건이 모두 포함되는지
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|plan              

### 관찰 포인트 — Z-order

- Z-order는 `product_name`과 `customer_id` **두 필드를 동시에 고려**하여 정렬했습니다
- 데모에서는 `target-file-size-bytes=8192`로 파일 수를 늘려 복합 필터 프루닝 가시성을 높였습니다
- 파티션당 파일이 1개면 파티션 프루닝 외에는 체감이 작아, Z-order 효과를 관찰하기 어렵습니다
- 복합 필터 쿼리(`WHERE A = x AND B = y`)에서 **프루닝 효과가 극대화**됩니다
- 단일 필드 필터에서는 Sort보다 효과가 떨어질 수 있지만, 여러 필드 조합 필터에서 강점을 발휘합니다

---
## 타겟 컴팩션과 고급 옵션

### 타겟 컴팩션 (Where 필터)

전체 테이블이 아닌 **특정 파티션이나 조건에 맞는 데이터만** 컴팩션할 수 있습니다:

```sql
CALL catalog.system.rewrite_data_files(
    table => 'my_table',
    strategy => 'binpack',
    where => 'order_date >= "2024-01-01"'
)
```

스트리밍 환경에서 최근 수집된 파티션만 주기적으로 컴팩션하면 효율적입니다.

### File Groups & Partial Progress

- **File Groups**: 컴팩션 대상 파일을 그룹으로 나눠 병렬 처리합니다
  - `max-file-group-size-bytes`: 그룹 최대 크기
  - `max-concurrent-file-group-rewrites`: 동시 처리 그룹 수

- **Partial Progress**: 파일 그룹 완료 시마다 새 스냅샷을 생성합니다
  - `partial-progress-enabled`: 부분 진행 활성화
  - `partial-progress-max-commits`: 최대 커밋 수
  - 장점: 다른 쿼리가 컴팩션 진행 중에도 이미 완료된 부분의 이점을 얻을 수 있음
  - 주의: 더 많은 스냅샷 = 더 많은 메타데이터 파일

---
## 전략 비교 요약

| 전략 | 속도 | 읽기 개선 | 적합 사례 |
|------|------|-----------|----------|
| **Binpack** | 가장 빠름 | 파일 수만 감소 | 스트리밍 SLA, 빠른 컴팩션 필요 시 |
| **Sort** | 중간 | 단일 필드 프루닝 | 자주 필터링하는 필드가 명확할 때 |
| **Z-order** | 중간 | 다중 필드 프루닝 | 복합 필터 쿼리가 빈번할 때 |

### 전략 선택 가이드

1. **"일단 파일 수를 줄이고 싶다"** → Binpack
2. **"특정 컬럼으로 자주 필터링한다"** → Sort
3. **"여러 컬럼 조합으로 필터링한다"** → Z-order
4. **"스트리밍 SLA가 빡빡하다"** → Binpack (속도 우선)

> 컴팩션은 일회성이 아닙니다. 데이터가 계속 쌓이면 주기적으로 실행해야 합니다.
> Airflow, Dagster 등 오케스트레이션 도구로 자동화하거나, 관리형 카탈로그(Tabular 등)의 자동 유지보수 기능을 활용하세요.

In [19]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
